In [20]:
import EEGExtract as eeg
import glob
import numpy as np

In [131]:
# Export data from the BCI Competition Dataset IV 2a dataset
# Code used from https://github.com/bregydoc/bcidatasetIV2a
class MotorImageryDataset:
    def __init__(self, dataset='A01T.npz'):
        if not dataset.endswith('.npz'):
            dataset += '.npz'

        self.data = np.load(dataset)

        self.Fs = 250 # 250Hz from original paper

        # keys of data ['s', 'etyp', 'epos', 'edur', 'artifacts']
        self.raw = self.data['s'].T
        self.events_type = self.data['etyp'].T
        self.events_position = self.data['epos'].T
        self.events_duration = self.data['edur'].T
        self.artifacts = self.data['artifacts'].T

        # Types of motor imagery
        self.mi_types = {769: 'left', 770: 'right', 771: 'foot',
                         772: 'tongue', 783: 'unknown', 1023:'rejected'}

    def get_trials_from_channel(self):

        # Channel default is C3
        startrial_code = 768
        starttrial_events = self.events_type == startrial_code
        idxs = [i for i, x in enumerate(starttrial_events[0]) if x]

        trials = []
        classes = []
        artifacts = []
        for ii, index in enumerate(idxs):
            type_e = self.events_type[0, index+1]
            if type_e not in self.mi_types.keys():
                continue
            class_e = self.mi_types[type_e]
            if class_e == 'unknown':
                continue
            classes.append(type_e-769)

            start = self.events_position[0, index] + 0.5 * self.Fs
            stop = start + self.events_duration[0, index]
            if stop < start + 2* self.Fs:
                print(stop,start + 2* self.Fs)
                raise '(MotorImageryDataset error): EEG is shorter than 2 sec'
            trial = self.raw[0:22, int(start):int(start + 2* self.Fs)]
            trials.append(trial)
            artifacts.append(self.artifacts[0,ii])
        return trials, classes, artifacts

In [132]:
trials = []
classes = []
artifacts = []
for file in glob.glob('../bcidatasetIV2a/*.npz'):
    datasetA1 = MotorImageryDataset(file)
    # trials contains the N valid trials, and clases its related class.
    tmp_trials, tmp_classes, tmp_artifacts = datasetA1.get_trials_from_channel()
    trials.extend(tmp_trials)
    classes.extend(tmp_classes)
    artifacts.extend(tmp_artifacts)

In [133]:
eegData = np.dstack(trials)

In [134]:
fs = 250

In [135]:
# eegData: 3D np array [chans x ms x epochs] 
eegData.shape

(22, 500, 2816)

In [136]:
feature_list = []

## Complexity Features

In [139]:
#Shannon Entropy
ShannonRes = eeg.shannonEntropy(eegData, bin_min=-200, bin_max=200, binWidth=2)

In [ ]:
#Tsalis Entropy (n=10)
tsalisRes = eeg.tsalisEntropy(eegData, bin_min=-200, bin_max=200, binWidth=2,list(range(1,10+1)))

In [9]:
# Subband Information Quantity
# delta (0.5–4 Hz)
eegData_delta = eeg.filt_data(eegData, 0.5, 4, fs)
ShannonRes_delta = eeg.shannonEntropy(eegData_delta, bin_min=-200, bin_max=200, binWidth=2)
# theta (4–8 Hz)
eegData_theta = eeg.filt_data(eegData, 4, 8, fs)
ShannonRes_theta = eeg.shannonEntropy(eegData_theta, bin_min=-200, bin_max=200, binWidth=2)
# alpha (8–12 Hz)
eegData_alpha = eeg.filt_data(eegData, 8, 12, fs)
ShannonRes_alpha = eeg.shannonEntropy(eegData_alpha, bin_min=-200, bin_max=200, binWidth=2)
# beta (12–30 Hz)
eegData_beta = eeg.filt_data(eegData, 12, 30, fs)
ShannonRes_beta = eeg.shannonEntropy(eegData_beta, bin_min=-200, bin_max=200, binWidth=2)
# gamma (30–100 Hz)
eegData_gamma = eeg.filt_data(eegData, 30, 100, fs)
ShannonRes_gamma = eeg.shannonEntropy(eegData_gamma, bin_min=-200, bin_max=200, binWidth=2)

In [ ]:
# Cepstrum Coefficients (n=2)
CepstrumRes = eeg.mfcc(eegData, fs,order=2)

In [10]:
# Lyapunov Exponent
LyapunovRes = eeg.lyapunov(eegData)

In [11]:
# Fractal Embedding Dimension
HiguchiFD_Res  = eeg.hFD(eegData[0,:,0],3)

/home/sarisadiya/Projects/ECC/EEGExtract/EEGExtract.py:256: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (p, r1, r2, s)=np.linalg.lstsq(x, L)


In [31]:
# Hjorth Mobility
# Hjorth Complexity
HjorthMob, HjorthComp = eeg.hjorthParameters(eegData)

In [32]:
# False Nearest Neighbor
FalseNnRes = eeg.falseNearestNeighbor(eegData)

In [ ]:
# ARMA Coefficients (n=2)
armaRes = eeg.arma(eegData,order=2)

## Category Features

In [7]:
# Median Frequency
medianFreqRes = eeg.medianFreq(eegData,fs)

In [15]:
# δ band Power
bandPwr_delta = eeg.bandPower(eegData, 0.5, 4, fs)
# θ band Power
bandPwr_theta = eeg.bandPower(eegData, 4, 8, fs)
# α band Power
bandPwr_alpha = eeg.bandPower(eegData, 8, 12, fs)
# β band Power
bandPwr_beta = eeg.bandPower(eegData, 12, 30, fs)
# γ band Power
bandPwr_gamma = eeg.bandPower(eegData, 30, 100, fs)

In [17]:
# Standard Deviation
std_res = eeg.eegStd(eegData)

In [42]:
# α/δ Ratio
ratio_res = eeg.eegRatio(eegData,fs)

In [124]:
# Regularity (burst-suppression)
regularity_res = eeg.eegRegularity(eegData,fs)

In [125]:
# Voltage < 5μ
volt05_res = eeg.eegVoltage(eegData,voltage=5)
# Voltage < 10μ
volt10_res = eeg.eegVoltage(eegData,voltage=10)
# Voltage < 20μ
volt20_res = eeg.eegVoltage(eegData,voltage=20)

In [126]:
# Diffuse Slowing
df_res = eeg.diffuseSlowing(eegData)

In [166]:
# Spikes
minNumSamples = int(70*fs/1000)
spikeNum_res = eeg.spikeNum(eegData,minNumSamples)

In [147]:
# Delta burst after Spike
deltaBurst_res = eeg.burstAfterSpike(eegData,eegData_delta,minNumSamples=7,stdAway = 3)

In [170]:
# Sharp spike
sharpSpike_res = eeg.shortSpikeNum(eegData,minNumSamples)

In [184]:
# Number of Bursts
numBursts_res = eeg.numBursts(eegData,fs)

In [204]:
# Burst length μ and σ
burstLenMean_res,burstLenStd_res = eeg.burstLengthStats(eegData,fs)

In [256]:
# Burst Band Power for δ
burstBandPwrAlpha = eeg.burstBandPowers(eegData, 0.5, 4, fs)

In [300]:
# Number of Suppressions
numSupps_res = eeg.numSuppressions(eegData,fs)

In [319]:
# Suppression length μ and σ
suppLenMean_res,suppLenStd_res = eeg.suppressionLengthStats(eegData,fs)

##    Connectivity features

In [361]:
# Coherence - δ
coherence_res = eeg.coherence(eegData,fs)

In [362]:
#import importlib
#importlib.reload(eeg)

In [507]:
feature_list = []
feature_list.append(ShannonRes)
feature_list.append(ShannonRes_delta)
feature_list.append(ShannonRes_theta)
#feature_list.append(ShannonRes_alpha)
#feature_list.append(ShannonRes_beta)
feature_list.append(ShannonRes_gamma)
feature_list.append(bandPwr_delta)
feature_list.append(bandPwr_theta)
#feature_list.append(bandPwr_alpha)
#feature_list.append(bandPwr_beta)
feature_list.append(bandPwr_gamma)
feature_list.append(std_res)
#feature_list.append(ratio_res)
feature_list.append(regularity_res)
feature_list.append(volt05_res)
feature_list.append(volt10_res)
feature_list.append(volt20_res)
feature_list.append(df_res)
feature_list.append(spikeNum_res)
feature_list.append(deltaBurst_res)
feature_list.append(sharpSpike_res)
feature_list.append(numBursts_res)
#feature_list.append(burstLenMean_res)
#feature_list.append(burstLenStd_res)
feature_list.append(numSupps_res)
#feature_list.append(coherence_res)

In [508]:
feature_arr  = np.vstack(feature_list).transpose()

In [509]:
feature_arr.shape

(2816, 396)

In [510]:
len(feature_list)

18

In [511]:
sum(artifacts) / len(artifacts)

0.17329545454545456

In [512]:
# https://pyod.readthedocs.io/en/latest/pyod.models.html
from pyod import models
from pyod.models import hbos,auto_encoder,lof,so_gaal,lscp,vae,abod,ocsvm,xgbod,pca

In [513]:
clf = hbos.HBOS(n_bins=17, alpha=0.07, tol=0.5,contamination=.15)
clf.fit(feature_arr)

HBOS(alpha=0.07, contamination=0.15, n_bins=17, tol=0.5)

In [523]:
from sklearn.metrics import confusion_matrix,cohen_kappa_score,f1_score

In [524]:
confusion_matrix(artifacts, clf.labels_)

array([[2018,  310],
       [ 375,  113]])

In [525]:
cohen_kappa_score(artifacts, clf.labels_)

0.10386299272297506

In [526]:
f1_score(artifacts, clf.labels_)

0.24807903402854006